# Querying and Visualizing Wikidata

Questions to Explore Later:

Should we make separate columns for every multiple entry - so Occupation 1, Occupation 2?
For now it seems fine to at least have a way to concatenate columns for appearance sake, then focus on visualization

Artists who are African-American who resided in Philadelphia
?artist wdt:P5008 wd:Q94124522 . = artists labeled by PMA as African-American artists in PMA Records
Compare with a query that just looks for Artists with associations as Black and somehow residing in Philadelphia

1) Create queries for all artists who resided in Philadelphia with identity information
2) Artists born in Philadelphia vs artists who have Philadelphia listed elsewhere in their history - focus on Residence Label vs Birthplace Label

NEW QUERY - take a shot at all Philly artists born in Philly with identity info


# Wikidata Questions
How do we want to visualize the data we've got so far?
Let's get a query of all artists in Philadelphia and visualize differences between types of artists . . .



# Sparqlwrapper Query Wikidata API

## Install Packages

In [ ]:
!pip install SPARQLWrapper
import sys
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
%load_ext google.colab.data_table 
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

## Set up SPARQL Query

In [ ]:
sparql.setQuery("""
SELECT
       ?artist ?artistLabel
       ?sexGenderLabel
       ?birthdayLabel
       ?birthPlaceLabel
       ?deathDateLabel
       ?deathPlaceLabel
       ?occupationLabel
       ?employerLabel
       ?workLocationLabel
       ?educationLabel
       ?residenceLabel
WHERE
{
       ?artist wdt:P5008 wd:Q94124522 .
       OPTIONAL {?artist wdt:P21 ?sexGender.}
       OPTIONAL {?artist wdt:P569 ?birthday.}
       OPTIONAL {?artist wdt:P19 ?birthPlace.}
       OPTIONAL {?artist wdt:P570 ?deathDate.}
       OPTIONAL {?artist wdt:P20 ?deathPlace.}
       OPTIONAL {?artist wdt:P106 ?occupation.}
       OPTIONAL {?artist wdt:P108 ?employer.}
       OPTIONAL {?artist wdt:P937 ?workLocation.}
       OPTIONAL {?artist wdt:P69 ?education.}
       OPTIONAL {?artist wdt:P551 ?residence.}
       SERVICE wikibase:label {bd:serviceParam wikibase:language "en".}
}
""")

In [ ]:
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
#results

## Create Dataframe and Wrangle Data

### Python Questions:

What other standardization of the data should I do? 
Multiple values for Education, Location, Occupation values
Top priorities are Artist and Location


In [ ]:
df = pd.io.json.json_normalize(results['results']['bindings'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


In [ ]:
cols = ['artistLabel.value', 
            'sexGenderLabel.value', 
            'birthdayLabel.value',
            'deathDateLabel.value',
            'birthPlaceLabel.value',
            'deathPlaceLabel.value',
            'residenceLabel.value',
            'educationLabel.value',
            'occupationLabel.value',
            'employerLabel.value',]

In [ ]:
df = df[cols]
df.head()

,artistLabel.value,sexGenderLabel.value,birthdayLabel.value,deathDateLabel.value,birthPlaceLabel.value,deathPlaceLabel.value,residenceLabel.value,educationLabel.value,occupationLabel.value,employerLabel.value
0,John Bunion Murray,male,1908-01-01T00:00:00Z,1988-01-01T00:00:00Z,Glascock County,Sandersville,NaN,NaN,artist,NaN
1,John Bunion Murray,male,1908-01-01T00:00:00Z,1988-09-18T00:00:00Z,Glascock County,Sandersville,NaN,NaN,artist,NaN
2,Claude Clark,male,1915-11-11T00:00:00Z,2001-04-21T00:00:00Z,Rockingham,Oakland,California,"University of California, Berkeley",artist,Works Progress Administration
3,Claude Clark,male,1915-11-11T00:00:00Z,2001-04-21T00:00:00Z,Rockingham,Oakland,California,"University of California, Berkeley",painter,Works Progress Administration
4,Claude Clark,male,1915-11-11T00:00:00Z,2001-04-21T00:00:00Z,Rockingham,Oakland,California,"University of California, Berkeley",visual artist,Works Progress Administration


In [ ]:
df.columns = ['name', 'gender', 'birthdate', 'deathdate', 'birthplace', 'deathplace', 'residence', 'education', 'occupation', 'employer']
df

,name,gender,birthdate,deathdate,birthplace,deathplace,residence,education,occupation,employer
0,John Bunion Murray,male,1908-01-01T00:00:00Z,1988-01-01T00:00:00Z,Glascock County,Sandersville,NaN,NaN,artist,NaN
1,John Bunion Murray,male,1908-01-01T00:00:00Z,1988-09-18T00:00:00Z,Glascock County,Sandersville,NaN,NaN,artist,NaN
2,Claude Clark,male,1915-11-11T00:00:00Z,2001-04-21T00:00:00Z,Rockingham,Oakland,California,"University of California, Berkeley",artist,Works Progress Administration
3,Claude Clark,male,1915-11-11T00:00:00Z,2001-04-21T00:00:00Z,Rockingham,Oakland,California,"University of California, Berkeley",painter,Works Progress Administration
4,Claude Clark,male,1915-11-11T00:00:00Z,2001-04-21T00:00:00Z,Rockingham,Oakland,California,"University of California, Berkeley",visual artist,Works Progress Administration
...,...,...,...,...,...,...,...,...,...,...
1524,Gordon Henderson,male,1957-03-19T00:00:00Z,NaN,NaN,NaN,Greenwich Village,Parsons School of Design,fashion designer,Calvin Klein Inc
1525,Michael D. Harris,male,1948-01-01T00:00:00Z,NaN,NaN,NaN,Atlanta,Yale University,writer,Emory University
1526,Michael D. Harris,male,1948-01-01T00:00:00Z,NaN,NaN,NaN,Atlanta,Yale University,artist,Emory University
1527,Michael D. Harris,male,1948-01-01T00:00:00Z,NaN,NaN,NaN,Atlanta,Yale University,university teacher,Emory University


In [ ]:
#df.dtypes

## Wrangle Data with Multiple Rows

In [ ]:
df1 = df.groupby(['name', 'gender'], dropna=False)['occupation'].apply(', '.join).reset_index()
dfNew

,name,gender,occupation
0,A.J. Smith,male,"professor, artist, printmaker"
1,Aaron Douglas,male,"illustrator, illustrator, painter, painter, mu..."
2,Akili Ron Anderson,male,"photographer, artist, painter, sculptor, stain..."
3,Alfred A. Smith,male,"artist, artist"
4,Alison Saar,female,"artist, illustrator, sculptor, photographer, a..."
...,...,...,...
198,William Majors,male,"artist, university teacher"
199,William Plummer,male,"inventor, cabinetmaker"
200,William T. Williams,male,"artist, painter, printmaker, artist, painter, ..."
201,Willie Birch,male,"artist, university teacher, artist, university..."


In [ ]:
dfNew['occupation'].drop_duplicates
dfNew

<bound method Series.drop_duplicates of 0                          professor, artist, printmaker
1      illustrator, illustrator, painter, painter, mu...
2      photographer, artist, painter, sculptor, stain...
3                                         artist, artist
4      artist, illustrator, sculptor, photographer, a...
                             ...                        
198                           artist, university teacher
199                               inventor, cabinetmaker
200    artist, painter, printmaker, artist, painter, ...
201    artist, university teacher, artist, university...
202                            visual artist, printmaker
Name: occupation, Length: 203, dtype: object>

In [ ]:
#df['name'] = df[['name','gender','birthdate','occupation']].groupby(['name'])['occupation'].apply(', '.join).reset_index()
#df[['name','gender','birthdate', 'occupation']].drop_duplicates()

# Other Cleaning

In [ ]:
#Clean Date Data to revise
#df['birthdayLabel.value'] = df['birthdayLabel.value'].astype(str).str[:4]
#df['deathDateLabel.value'] = df['deathDateLabel.value'].astype(str).str[:4]
#df['birthdayLabel.value'] = df['birthdayLabel.value'].astype(int)
#df['deathDateLabel.value'] = df['deathDateLabel.value'].astype(int)
#df

In [ ]:
#Need to set astype like this: 
df.set_index('capitalLabel', inplace=True)
df = df.astype({'population': float, 'age': float,
                'capital_lon': float, 'capital_lat': float, 
                'birth_place_lon': float, 'birth_place_lat': float})

## Visualize significant details

In [ ]:
df['birthdayLabel.value'].value_counts()[:10]

KeyError: ignored

In [ ]:
df['workLocationLabel.value'].value_counts()[:10]

In [ ]:
df['age'] = df['deathDateLabel.value'] - df['birthdayLabel.value']

In [ ]:
import seaborn
df.plot(x = 'artistLabel.value', 
        y = 'age', 
        kind='barh', 
        title='Artist Deaths')

# Sample Visualization

https://towardsdatascience.com/where-do-mayors-come-from-querying-wikidata-with-python-and-sparql-91f3c0af22e2

In [ ]:
# Adapt to our dataset:

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')plt.figure(figsize=(16, 12))
for i, label in enumerate(['population', 'medianIncome', 'area', 'age']):
    plt.subplot(2, 2, i + 1)
    df_plot = df[label].sort_values().dropna()
    df_plot.plot(kind='barh', color='C0', ax=plt.gca());
    plt.ylabel('')
    plt.xticks(rotation=30)
    plt.title(label.capitalize())
    plt.ticklabel_format(style='plain', axis='x')
plt.tight_layout()

In [ ]:
#Alternative viz more advanced

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
df['age'].sort_values().plot(kind='barh', color='C0', title='Mayors Age')
plt.ylabel('')
plt.subplot(1, 2, 2)
df['population'].sort_values().plot(kind='barh', color='C0', title='Population')
plt.ylabel('')
plt.ticklabel_format(style='plain', axis='x')
plt.tight_layout()

In [ ]:
from geopy.distance import distancecoordinates = df[['capital_lon', 'capital_lat', 
                  'birth_place_lon', 'birth_place_lat']]
df['distance'] = [distance((lat0, lon0), (lat1, lon1)).m 
                for lon0, lat0, lon1, lat1 in coordinates.values]df['distance'].sort_values().plot(
    kind='barh', color='C0', logx=True)
plt.xlabel('Distance (m)')
plt.ylabel('');

# QWikidata - Functional

In [ ]:
#https://qwikidata.readthedocs.io/en/stable/readme.html
!pip install qwikidata

from qwikidata.sparql  import return_sparql_query_results

In [ ]:
#reframe our query this way
query_string = """
        SELECT $WDid
         WHERE {
          ?WDid (wdt:P279)* wd:Q4022
        }"""

In [ ]:
query_string = """
SELECT
       ?artist ?artistLabel
       ?sexGenderLabel
       ?birthdayLabel
       ?birthPlaceLabel
       ?deathDateLabel
       ?deathPlaceLabel
       ?occupationLabel
       ?employerLabel
       ?workLocationLabel
       ?educationLabel
       ?residenceLabel
WHERE
{
       ?artist wdt:P5008 wd:Q94124522 .
       OPTIONAL {?artist wdt:P21 ?sexGender.}
       OPTIONAL {?artist wdt:P569 ?birthday.}
       OPTIONAL {?artist wdt:P19 ?birthPlace.}
       OPTIONAL {?artist wdt:P570 ?deathDate.}
       OPTIONAL {?artist wdt:P20 ?deathPlace.}
       OPTIONAL {?artist wdt:P106 ?occupation.}
       OPTIONAL {?artist wdt:P108 ?employer.}
       OPTIONAL {?artist wdt:P937 ?workLocation.}
       OPTIONAL {?artist wdt:P69 ?education.}
       OPTIONAL {?artist wdt:P551 ?residence.}
       SERVICE wikibase:label {bd:serviceParam wikibase:language "en".}
}
"""

In [ ]:
query_string="""
SELECT
       ?artist ?artistLabel
       
WHERE
{
       ?artist wdt:P5008 wd:Q94124522. 
       SERVICE wikibase:label {bd:serviceParam wikibase:language "[AUTO_LANGUAGE]"
}"""

In [ ]:
res = return_sparql_query_results(query_string)

In [ ]:
print(res)

In [ ]:
for row in res["results"]["bindings"]:
   print(row["artistLabel"]["value"])

In [ ]:
import pandas as pd

results_df = pd.io.json.json_normalize(res['results']['bindings'])

In [ ]:
results_df[['artist.value', 'artistLabel.value']]

In [ ]:
results_df

# Alternative API Query Method

In [ ]:
import pandas as pd
import json
from SPARQLWrapper import SPARQLWrapper, JSON

In [ ]:
def get_sparql_dataframe(service, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    sparql = SPARQLWrapper(service)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

In [ ]:
wds = "https://query.wikidata.org/sparql"

In [ ]:
rq = """
SELECT
       ?artist ?artistLabel
       ?sexGenderLabel
       ?birthdayLabel
       ?birthPlaceLabel
       ?deathDateLabel
       ?deathPlaceLabel
       ?occupationLabel
       ?employerLabel
       ?workLocationLabel
       ?educationLabel
       ?residenceLabel
WHERE
{
       ?artist wdt:P5008 wd:Q94124522 .
       OPTIONAL {?artist wdt:P21 ?sexGender.}
       OPTIONAL {?artist wdt:P569 ?birthday.}
       OPTIONAL {?artist wdt:P19 ?birthPlace.}
       OPTIONAL {?artist wdt:P570 ?deathDate.}
       OPTIONAL {?artist wdt:P20 ?deathPlace.}
       OPTIONAL {?artist wdt:P106 ?occupation.}
       OPTIONAL {?artist wdt:P108 ?employer.}
       OPTIONAL {?artist wdt:P937 ?workLocation.}
       OPTIONAL {?artist wdt:P69 ?education.}
       OPTIONAL {?artist wdt:P551 ?residence.}
       SERVICE wikibase:label {bd:serviceParam wikibase:language "en".}
}
"""

In [ ]:
df = get_sparql_dataframe(wds, rq)

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df(occupationLabel)